In [1]:
import json
import os
os.chdir('..')
import pandas as pd

In [4]:
with open('data/raw/bhajans_info.json', mode ='r') as file:
    bhajan_list = json.loads(file.read())

In [6]:
os.getcwd()

'C:\\Users\\KumarArv\\Documents\\Personal Projects\\raaga'

In [9]:
bhajan_df = pd.DataFrame(bhajan_list)
bhajan_df.dropna(inplace=True)
bhajan_df.reset_index(inplace=True, drop=True)
bhajan_df.to_csv('data/processed/bhajans_info_cleaned.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/bhajans_info_cleaned.csv'

In [8]:
os.listdir()

['.idea', 'bhajancrawler', 'data', 'noteboooks', 'results', 'src']

In [ ]:
bhajan_df.to_csv()